# Imports

In [ ]:
import packages
from configs import settings, const, components
from configs.settings import logger
import asyncio, os, time, yaml, json, datetime, copy, random
from typing import Any, AsyncGenerator, Generator, Callable, Literal, Optional, TypeAlias, Union
from tqdm import tqdm
from pprint import pprint

from toolkit.llm.llama_index import (
	agents, cores, deploys as dpls, evaluation, messages, models, 
	observability, types, utils as utils_llama_index, workflows as wfs
)
from toolkit.llm.llama_index.data import loading, querying, storing

from features.agents.car.tools import VehicleDB
from features.agents.tools import map

from toolkit.utils import utils, typer as t
from toolkit.utils.llm import measure_performance, main as utils_llm
from toolkit.utils.utils import rp_print

# Dev

## Starter

### Basic

In [ ]:
class MyWorkflow(wfs.Workflow):
	@wfs.step
	async def my_step(self, ev: wfs.StartEvent) -> wfs.StopEvent:
		return wfs.StopEvent(result="Hello, world!")

async def main():
	workflow = MyWorkflow(timeout=10, verbose=False)
	result = await workflow.run()
	pprint(result)

# wfs.draw_all_possible_flows(MyWorkflow)

asyncio.run(main())

In [ ]:
class LlmGenerator(wfs.Workflow):
	@wfs.step()
	async def generate(
		self, ctx: wfs.Context, ev: wfs.StartEvent,
	) -> wfs.StopEvent:
		llm = cores.Settings.llm
		response = await llm.acomplete(ev.get("query"))
		return wfs.StopEvent(result=str(response))

async def main():
	workflow = LlmGenerator()
	result = await workflow.run(query="What's LlamaIndex?")
	print(result)

asyncio.run(main())

In [ ]:
import random

class EventFailed(wfs.Event):
	error: str

class EventQuery(wfs.Event):
	query: str

class MyWorkflow(wfs.Workflow):
	@wfs.step()
	async def setup(
		self, ctx: wfs.Context, ev: wfs.StartEvent
	) -> EventQuery:
		if not hasattr(ev, "data"):
			await ctx.set("data", ["val1", "val2", "val3"])

		return EventQuery(query=ev.get("query"))
	
	@wfs.step()
	async def answer_query(
		self, ctx: wfs.Context, ev: EventQuery,
	) -> EventFailed | wfs.StopEvent:
		query = ev.get("query")
  
		data = await ctx.get("data") if hasattr(ev, "data") else None
		result = data[random.randint(0, len(data))] if data else None
	
		random_number = random.randint(0, 1)

		if random_number == 0:
			return EventFailed(error="Failed to answer user query.")
		else:
			return wfs.StopEvent(result=f"The answer to your query is {result}.")
		
	@wfs.step()
	async def improve_query(
		self, ctx: wfs.Context, ev: EventFailed,
	) -> EventQuery | wfs.StopEvent:
		random_number = random.randint(0, 1)

		if random_number == 0:
			return EventQuery(query="Here's a better query.")
		else:
			return wfs.StopEvent(result="Your query can't be fixed.")

async def main():
	workflow = MyWorkflow()
	result = await workflow.run(query="What's LlamaIndex?")
	print(result)

asyncio.run(main())

In [ ]:
import random

class EventLoop(wfs.Event):
	output_loop: str

class EventSetupContext(wfs.Event):
	query: str

class EventFirst(wfs.Event):
	output_first: str

class EventSecond(wfs.Event):
	output_second: str
	response: str

class EventBranchA1(wfs.Event):
	payload: str

class EventBranchA2(wfs.Event):
	payload: str

class EventBranchB1(wfs.Event):
  payload: str

class EventBranchB2(wfs.Event):
	payload: str

class MyWorkflow(wfs.Workflow):
	@wfs.step(num_workers=4)
	async def start(
		self, ctx: wfs.Context, 
  	ev: wfs.StartEvent,
	) -> EventSetupContext | EventFirst:
		try:
			db = await ctx.get("some_database", default=None)
		except:
			await ctx.set("some_database", None)
			db = await ctx.get("some_database", default=None)
   
			if db is None:
				print(f"No database found, setting up...")
				return EventSetupContext(query="Setup")

		print(f"Database found: {db}")
		return EventFirst(output_first="First step output")
	
	@wfs.step(num_workers=4)
	async def setup(
		self, ctx: wfs.Context, 
  	ev: EventSetupContext,
	) -> wfs.StartEvent:
		await ctx.set("some_database", [1, 2, 3])
		print(f"Database setup completed.")
		return wfs.StartEvent(query=ev.query)
 
	@wfs.step(num_workers=4)
	async def step_one(
   	self, ctx: wfs.Context, 
    ev: EventFirst | EventLoop
  ) -> EventSecond | EventLoop:
		ctx.write_event_to_stream(wfs.Event(msg="Step one is happening ..."))
		if random.randint(0, 1) == 0:
			print("Bad thing happened")
			return EventLoop(output_loop="Looping back to step one.")
		else:
			print("Good thing happened")
			# llm = models.OpenAI(model="gpt-3.5-turbo")
			# generator = await llm.astream_complete("Tell me a joke")
			# async for response in generator:
			# 	ctx.write_event_to_stream(wfs.Event(msg=response.delta))
			response = None
			return EventSecond(
				output_second="First step completed.",
				response=str(response)
			)
		# print(ev.input_first)
		# return EventFirst(output_first="First step completed.")

	@wfs.step(num_workers=4)
	async def step_two(
   self, ctx: wfs.Context,
   ev: EventSecond
  ) -> EventBranchA1 | EventBranchB1:
		ctx.write_event_to_stream(wfs.Event(msg="Step two is happening ..."))
		
		if random.randint(0, 1) == 0:
			return EventBranchA1(payload="Branch A")
		else:
			return EventBranchB1(payload="Branch B")
 
	@wfs.step(num_workers=4)
	async def step_branch_a1(
   self, ctx: wfs.Context,
   ev: EventBranchA1
  ) -> EventBranchA2:
		print(ev.payload)
		return EventBranchA2(payload="Branch A2")

	@wfs.step(num_workers=4)
	async def step_branch_a2(
   self, ctx: wfs.Context,
   ev: EventBranchA2
  ) -> wfs.StopEvent:
		print(ev.payload)
		return wfs.StopEvent(result="Workflow completed.")

	@wfs.step(num_workers=4)
	async def step_branch_b1(
   self, ctx: wfs.Context,
   ev: EventBranchB1
  ) -> EventBranchB2:
		print(ev.payload)
		return EventBranchB2(payload="Branch B2")

	@wfs.step(num_workers=4)
	async def step_branch_b2(
   self, ctx: wfs.Context,
   ev: EventBranchB2
  ) -> wfs.StopEvent:
		print(ev.payload)
		return wfs.StopEvent(result="Workflow completed.")

async def main():
	workflow = MyWorkflow(timeout=10, verbose=False)
	result = await workflow.run(query="Starting the workflow.")
	pprint(result)

wfs.draw_all_possible_flows(MyWorkflow)
asyncio.run(main())

### Concurrent/Parallel

In [ ]:
import asyncio
import random
from pprint import pprint
from llama_index.core.workflow import (
    step,
    Context,
    Workflow,
    Event,
    StartEvent,
    StopEvent,
)

# Event definitions
class EventSetup(Event):
    error: bool

class EventInput(Event):
    input: str

class EventQuery(Event):
    query: str

class ProcessEvent(Event):
    data: str

class ResultEvent(Event):
    result: str

class ParallelWorkflow(Workflow):
    @step()
    async def setup(self, ctx: Context, ev: StartEvent) -> EventSetup:
        if not hasattr(self, "setup") or not self.setup:
            self.setup = True
            print("Setup ✅")
        return EventSetup(error=False)

    @step()
    async def collect_input(self, ctx: Context, ev: StartEvent) -> EventInput:
        if hasattr(ev, "input"):
            print("Input ✅")
            return EventInput(input=ev.input)

    @step()
    async def parse_query(self, ctx: Context, ev: StartEvent) -> EventQuery:
        if hasattr(ev, "query"):
            print("Query ✅")
            return EventQuery(query=ev.query)

    @step()
    async def run_query(self, ctx: Context, ev: EventInput | EventSetup | EventQuery) -> ProcessEvent | None:
        results_events = ctx.collect_events(ev, [EventInput, EventSetup, EventQuery])
        if not results_events:
            print("Not enough events yet")
            return None

        my_query = results_events[2].query
        my_input = results_events[0].input

        print("Events collected 🫡")
        print(results_events)

        # Generate multiple ProcessEvents based on the input and query
        data_list = [f"{my_query} on {my_input}-{i}" for i in range(3)]
        await ctx.set("num_to_collect", len(data_list))
        for item in data_list:
            ctx.send_event(ProcessEvent(data=item))

        return None

    @step(num_workers=3)
    async def process_data(self, ctx: Context, ev: ProcessEvent) -> ResultEvent:
        # Simulate some time-consuming processing
        await asyncio.sleep(random.randint(1, 2))
        result = f"Processed: {ev.data}"
        print(f"Completed processing: {ev.data}")
        return ResultEvent(result=result)

    @step()
    async def combine_results(self, ctx: Context, ev: ResultEvent) -> StopEvent | None:
        num_to_collect = await ctx.get("num_to_collect")
        results = ctx.collect_events(ev, [ResultEvent] * num_to_collect)
        if results is None:
            return None

        combined_result = ", ".join([event.result for event in results])
        return StopEvent(result=f"Final result: {combined_result}")

async def main():
    workflow = ParallelWorkflow(timeout=20, verbose=False)
    result = await workflow.run(input="My Input", query="My Question")
    pprint(result)

if __name__ == "__main__":
    asyncio.run(main())

### Nested

In [ ]:
class WorkflowReflection(wfs.Workflow):
	@wfs.step(num_workers=4)
	async def sub_start(
		self, ctx: wfs.Context,
		ev: wfs.StartEvent,
	) -> wfs.StopEvent:
		print(f"Running reflection query: {ev.query}")
		return wfs.StopEvent(result="Reflection result")

class WorkflowReflectionDefault(wfs.Workflow):
	@wfs.step(num_workers=4)
	async def sub_start(
		self, ctx: wfs.Context,
		ev: wfs.StartEvent,
	) -> wfs.StopEvent:
		print("Doing default reflection")
		return wfs.StopEvent(result="Reflection result")

class EventStep1(wfs.Event):
	query: str

class WorkflowMain(wfs.Workflow):
	@wfs.step(num_workers=4)
	async def start(
		self, ctx: wfs.Context,
		ev: wfs.StartEvent,
		workflow_reflection: wfs.Workflow,
	) -> EventStep1:
		print("Need to run reflection")
		result = await workflow_reflection.run(query="Reflection query")
		return EventStep1(query=result)

	@wfs.step(num_workers=4)
	async def step_1(
		self, ctx: wfs.Context,
		ev: EventStep1,
	) -> wfs.StopEvent:
		print(f"Running query: {ev.query}")
		return wfs.StopEvent(result=ev.query)

workflow = WorkflowMain(timeout=10, verbose=False)
workflow.add_workflows(workflow_reflection=WorkflowReflection())

result = await workflow.run(query="Initial query")
print(result)

### Human-in-the-loop

In [ ]:
# Import necessary libraries and modules
from typing import List, Optional
from pydantic import BaseModel, Field
from llama_index.core.workflow import Context, Event, StartEvent, StopEvent, Workflow, step
from llama_index.llms.openai import OpenAI
from llama_index.core.prompts import PromptTemplate
import uuid

# Define the data model for scenarios
class Scenario(BaseModel):
    """Data model for generating scenarios in the workflow."""
    description: str = Field(description="The description of the current scenario.")
    options: List[str] = Field(default=[], description="The list of options available for the current scenario.")

# Define the Block class to represent a single step in the workflow
class Block(BaseModel):
    id_: str = Field(default_factory=lambda: str(uuid.uuid4()))  # Unique identifier for each block
    scenario: Scenario  # The scenario presented in this block
    choice: Optional[str] = None  # The choice made by the user, if any

    def __str__(self):
        """String representation of the Block for logging and display purposes."""
        return f"BLOCK\n===\nDESCRIPTION: {self.scenario.description}\nOPTIONS: {', '.join(self.scenario.options)}\nCHOICE: {self.choice or ''}"

# Define custom events for the workflow
class NewBlockEvent(Event):
    """Event triggered when a new block is created."""
    block: Block

class HumanChoiceEvent(Event):
    """Event triggered when a human makes a choice."""
    block_id: str

# Template for generating new scenarios based on previous ones
SCENARIO_GENERATION_TEMPLATE = """
You are assisting in an interactive decision-making process. Based on the previous scenarios and choices, generate the next scenario with a description and a set of options.

PREVIOUS SCENARIOS:
---
{running_scenarios}

Generate the next scenario and options. If there are no previous scenarios, start with an interesting initial scenario.

Use the provided data model to structure your output.
"""

# Main workflow class
class InteractiveDecisionMakingWorkflow(Workflow):
    def __init__(self, max_steps: int = 3, **kwargs):
        """
        Initialize the workflow.
        
        :param max_steps: Maximum number of steps in the workflow
        :param kwargs: Additional keyword arguments
        """
        super().__init__(**kwargs)
        self.llm = OpenAI()  # Initialize the language model
        self.max_steps = max_steps

    @step
    async def create_scenario(self, ctx: Context, ev: StartEvent | HumanChoiceEvent) -> NewBlockEvent | StopEvent:
        """
        Create a new scenario based on previous scenarios and choices.
        
        :param ctx: The context of the workflow
        :param ev: The triggering event (either StartEvent or HumanChoiceEvent)
        :return: Either a NewBlockEvent with the new scenario or a StopEvent if max steps reached
        """
        blocks = await ctx.get("blocks", [])
        running_scenarios = "\n".join(str(b) for b in blocks)

        if len(blocks) < self.max_steps:
            # Generate a new scenario using the language model
            new_scenario = self.llm.structured_predict(
                Scenario,
                PromptTemplate(SCENARIO_GENERATION_TEMPLATE),
                running_scenarios=running_scenarios,
            )
            new_block = Block(scenario=new_scenario)
            blocks.append(new_block)
            await ctx.set("blocks", blocks)
            return NewBlockEvent(block=new_block)
        else:
            # If max steps reached, stop the workflow
            return StopEvent(result=blocks)

    @step
    async def prompt_human(self, ctx: Context, ev: NewBlockEvent) -> HumanChoiceEvent:
        """
        Present the current scenario to the human and collect their choice.
        
        :param ctx: The context of the workflow
        :param ev: The NewBlockEvent containing the current scenario
        :return: A HumanChoiceEvent with the user's choice
        """
        block = ev.block

        # Prepare the prompt for the human
        human_prompt = f"\n===\n{block.scenario.description}\n\n"
        human_prompt += "Choose an option:\n\n"
        human_prompt += "\n".join(f"{i+1}. {option}" for i, option in enumerate(block.scenario.options))
        human_prompt += "\n\nEnter the number of your choice: "
        
        # Loop until a valid choice is made
        while True:
            human_input = input(human_prompt)
            try:
                choice_index = int(human_input) - 1
                if 0 <= choice_index < len(block.scenario.options):
                    block.choice = block.scenario.options[choice_index]
                    break
                else:
                    print("Invalid choice. Please try again.")
            except ValueError:
                print("Please enter a valid number.")

        # Update the context with the human's choice
        blocks = await ctx.get("blocks")
        blocks[-1] = block
        await ctx.set("blocks", blocks)

        return HumanChoiceEvent(block_id=block.id_)

# Example usage of the workflow
async def main():
    # Create and run the workflow
    workflow = InteractiveDecisionMakingWorkflow(max_steps=3)
    result = await workflow.run()
    
    # Print the final decision path
    print("\nFinal Decision Path:")
    for block in result:
        print(f"\nScenario: {block.scenario.description}")
        print(f"Choice: {block.choice}")

# Entry point of the script
if __name__ == "__main__":
    import asyncio
    asyncio.run(main())  # Run the main function in an async context

### Reflection

#### Structured Outputs (JSON-Extraction)

In [ ]:
from typing import Type
from pydantic import BaseModel

class EventDataExtracted(wfs.Event):
    output: str
    passage: str

class EventExtractionError(wfs.Event):
    error: str
    wrong_output: str
    passage: str

EXTRACTION_PROMPT = """
Context information is below:
---------------------
{passage}
---------------------

Given the context information and not prior knowledge, create a JSON object from the information in the context.
The JSON object must follow the JSON schema:
{schema}

"""

REFLECTION_PROMPT = """
You already created this output previously:
---------------------
{wrong_answer}
---------------------

This caused the JSON decode error: {error}

Try again, the response must contain only valid JSON code. Do not add any sentence before or after the JSON object.
Do not repeat the schema.
"""

class GenericReflectionWorkflow(wfs.Workflow):
    def __init__(
      self, 
      model_class: typer.Type[typer.BaseModel], 
      max_retries: int = 3, **kwargs
    ):
        super().__init__(**kwargs)
        self.model_class = model_class
        self.max_retries = max_retries
        self.current_retries = 0

    @wfs.step
    async def extract(
        self, ctx: wfs.Context, ev: wfs.StartEvent | EventExtractionError
    ) -> wfs.StopEvent | EventDataExtracted:
        if self.current_retries >= self.max_retries:
            return wfs.StopEvent(result="Max retries reached")
        else:
            self.current_retries += 1

        if isinstance(ev, wfs.StartEvent):
            passage = ev.get("passage")
            if not passage:
                return wfs.StopEvent(result="Please provide some text in input")
            reflection_prompt = ""
        elif isinstance(ev, EventExtractionError):
            passage = ev.passage
            reflection_prompt = REFLECTION_PROMPT.format(
                wrong_answer=ev.wrong_output, error=ev.error
            )

        llm = cores.Settings.llm
        prompt = EXTRACTION_PROMPT.format(
            passage=passage, schema=self.model_class.schema_json()
        )
        if reflection_prompt:
            prompt += reflection_prompt

        output = await llm.acomplete(prompt)

        return EventDataExtracted(output=str(output), passage=passage)

    @wfs.step
    async def validate(
        self, ev: EventDataExtracted
    ) -> wfs.StopEvent | EventExtractionError:
        try:
            self.model_class.model_validate_json(ev.output)
        except Exception as e:
            print("Validation failed, retrying...")
            return EventExtractionError(
                error=str(e), wrong_output=ev.output, passage=ev.passage
            )

        return wfs.StopEvent(result=ev.output)

# Example usage
class Person(typer.BaseModel):
    name: str
    age: int
    occupation: str

class PersonCollection(typer.BaseModel):
    people: list[Person]

async def main():
    workflow = GenericReflectionWorkflow(
        model_class=PersonCollection,
        max_retries=5,
        timeout=120,
        verbose=True
    )

    result = await workflow.run(
        passage="John is a 35-year-old software engineer. His colleague Sarah, who is 42, works as a project manager."
    )
    print(result)

if __name__ == "__main__":
    import asyncio
    asyncio.run(main())

### Hybrid

- Maintaining state
- Branches, loops
- Human-feedback
- Concurrent execution
- ...

In [ ]:
from toolkit.llm.llama_index import (
    cores, workflows as wfs, messages,
)
from toolkit.utils.llm import main as utils_llm

from typing import List, Optional

class EventEmptyJoke(wfs.Event):
    pass

class EventJoke(wfs.Event):
    joke: str

class EventMultiJoke(wfs.Event):
    jokes: List[str]

class EventBestJoke(wfs.Event):
    joke: str
    reason: str

class InputRequiredEvent(wfs.Event):
    prefix: str

class HumanResponseEvent(wfs.Event):
    response: str

class EventRating(wfs.Event):
    rating: int

class EventGatherJokes(wfs.Event):
    pass

class FlowJoke(wfs.Workflow):
    @wfs.step
    async def dispatch_joke_generation(
        self, ctx: wfs.Context, ev: wfs.StartEvent,
    ) -> EventGatherJokes | EventEmptyJoke:
        topic = ev.get("topic")
        await ctx.set(key="topic", value=topic)
        num_jokes = 3  # Generate 3 jokes

        for _ in range(num_jokes):
            ctx.send_event(EventEmptyJoke()) # Concurrent execution

        return EventGatherJokes()

    @wfs.step(
      retry_policy=wfs.ConstantDelayRetryPolicy(delay=5, maximum_attempts=3),
			num_workers=4,
    )
    async def generate_joke(
        self, ctx: wfs.Context, ev: EventEmptyJoke,
    ) -> EventJoke:
        topic = await ctx.get(key="topic")
        prompt = f"Write your best joke about {topic}"
        response = await cores.Settings.llm.acomplete(prompt)
        return EventJoke(joke=str(response))

    @wfs.step
    async def gather_jokes(
        self, ctx: wfs.Context, ev: EventGatherJokes | EventJoke,
    ) -> Optional[EventMultiJoke]:
        jokes = ctx.collect_events(ev, [EventJoke, EventJoke, EventJoke])
        if jokes is None:
            return None
        return EventMultiJoke(jokes=[joke.joke for joke in jokes])

    @wfs.step(retry_policy=wfs.ConstantDelayRetryPolicy(delay=5, maximum_attempts=3))
    async def select_best_joke(
        self, ctx: wfs.Context, ev: EventMultiJoke,
    ) -> EventBestJoke:
        jokes = ev.jokes
        prompt = """\
        Select the best joke from these options and explain why it's the best:

        {jokes}

        Please provide your response in JSON format as follows:

				```json
        {
            "best_joke": "BEST_JOKE",
            "reason": "REASON"
        }
        ```

        Just return the json, no other text.
        """
        prompt = messages.PromptTemplate(prompt)
        
        response = await cores.Settings.llm.achat([
            messages.ChatMessage(role="user", content=prompt.format(jokes=jokes)) # type: ignore
        ])
        response = str(response).split("assistant: ")[1]
        response = utils_llm.parse_json(response)

        best_joke = response["best_joke"]
        reason = response["reason"]

        return EventBestJoke(joke=best_joke, reason=reason)

    @wfs.step
    async def request_user_rating(
        self, ctx: wfs.Context, ev: EventBestJoke,
    ) -> InputRequiredEvent:
        joke = ev.joke
        return InputRequiredEvent(prefix=f"Please rate this joke from 1 to 5:\n\n{joke}\n\nYour rating: ")

    @wfs.step
    async def handle_human_response(
        self, ctx: wfs.Context, ev: InputRequiredEvent,
    ) -> HumanResponseEvent:
        # Get real user input
        user_input = input(ev.prefix)
        return HumanResponseEvent(response=user_input)

    @wfs.step
    async def process_user_rating(
        self, ctx: wfs.Context, ev: HumanResponseEvent,
    ) -> EventRating | InputRequiredEvent:
        try:
            rating = int(ev.response)
            if 1 <= rating <= 5:
                return EventRating(rating=rating)
            else:
                raise ValueError
        except ValueError:
            return InputRequiredEvent(prefix="Invalid input. Please enter a number between 1 and 5: ")

    @wfs.step(retry_policy=wfs.ConstantDelayRetryPolicy(delay=5, maximum_attempts=3))
    async def prepare_critique(
        self, ctx: wfs.Context, ev: EventBestJoke | EventRating,
    ) -> Optional[wfs.StopEvent]:
        data = ctx.collect_events(ev, [EventBestJoke, EventRating])
        
        if data is None:
            return None
        
        best_joke_event, rating_event = data
        
        joke = best_joke_event.joke
        reason = best_joke_event.reason
        rating = rating_event.rating
        
        prompt = f"Give a thorough analysis and critique of the following joke (rated {rating}/5 by a user):\n\nJoke: {joke}\n\nThis joke was selected as the best because: {reason}"
        response = await cores.Settings.llm.acomplete(prompt)
        return wfs.StopEvent(result=str(response))

# Example usage
workflow = FlowJoke(timeout=60, verbose=True)

wfs.draw_all_possible_flows(workflow)

result = await workflow.run(topic="pirates")
print(str(result))

### Agents

In [ ]:
class FieldsStateAgent(t.EnumCustom):
	USER_QUERY = t.auto()
	CURRENT_REASONING = t.auto()
	
class StateAgent(t.BaseModel):
	user_query: str = t.Field(description="User Input", default="")
	current_reasoning: list = t.Field(description="Current Agent Reasoning", default=[])

state_agent = StateAgent(
	user_query="",
	current_reasoning=[],
)


#### Function Calling

In [ ]:
class EvInput(wfs.Event):
	input: list[messages.ChatMessage]

class EvToolCall(wfs.Event):
	tool_calls: list[agents.ToolSelection]

class EvToolOutput(wfs.Event):
	output: agents.ToolOutput

class AgentFunctionCalling(wfs.Workflow):
	def __init__(
		self,
		llm: models.FunctionCallingLLM = cores.Settings.llm,
		tools: t.List[agents.BaseTool] = [],
		*args: t.Any,
		**kwargs: t.Any,
	) -> None:
		super().__init__(*args, **kwargs)
  
		self.tools = tools
		self.tools_dict = {tool.metadata.get_name(): tool for tool in self.tools}

		self.llm = llm
		assert self.llm.metadata.is_function_calling_model

		self.memory = messages.ChatMemoryBuffer.from_defaults(llm=self.llm)
		self.sources = []
	
	@wfs.step()
	async def prepare_chat_history(
		self, ev: wfs.StartEvent,
	) -> EvInput:
		self.sources.clear()

		user_query = ev.get(
    	FieldsStateAgent.USER_QUERY, 
    	t.get_field_default(StateAgent, FieldsStateAgent.USER_QUERY),
    )
		user_msg = messages.ChatMessage(
    	role=messages.MessageRole.USER, content=user_query,
    )
		self.memory.put(user_msg)

		chat_history = self.memory.get()
		return EvInput(input=chat_history)

	@wfs.step()
	async def handle_llm_input(
		self, ev: EvInput,
	) -> EvToolCall | wfs.StopEvent:
		chat_history = ev.input

		response = await self.llm.achat_with_tools(
			tools=self.tools, chat_history=chat_history,
		)
		self.memory.put(response.message)

		tool_calls = self.llm.get_tool_calls_from_response(
			response=response, error_on_no_tool_call=False,
		)

		if not tool_calls:
			return wfs.StopEvent(
				result={
					"response": response,
					"sources": [*self.sources],
				}
			)
		else:
			return EvToolCall(tool_calls=tool_calls)
		
	@wfs.step()
	async def handle_tool_calls(
		self, ev: EvToolCall
	) -> EvInput:
		tool_calls = ev.tool_calls

		tool_msgs = []

		for tool_call in tool_calls:
			tool = self.tools_dict.get(tool_call.tool_name)
			additional_kwargs = {
				"tool_call_id": tool_call.tool_id,
				"name": tool.metadata.get_name(),
			}

			if not tool:
				tool_msgs.append(
					messages.ChatMessage(
						role=messages.MessageRole.TOOL,
						content=f"Tool {tool_call.tool_name} does not exist",
						additional_kwargs=additional_kwargs,
					)
				)
				continue

			try:
				tool_output = tool(**tool_call.tool_kwargs)
				self.sources.append(tool_output)
				tool_msgs.append(
					messages.ChatMessage(
						role=messages.MessageRole.TOOL,
						content=tool_output.content,
						additional_kwargs=additional_kwargs,
					)
				)
			except Exception as e:
				tool_msgs.append(
					messages.ChatMessage(
						role=messages.MessageRole.TOOL,
						content=f"Encountered error in tool  call: {e}",
						additional_kwargs=additional_kwargs,
					)
				)

		for msg in tool_msgs:
			self.memory.put(msg)
		
		chat_history = self.memory.get()
		return EvInput(input=chat_history)

In [ ]:
def add(x: int, y: int) -> int:
	"""Useful function to add two numbers."""
	return x + y


def multiply(x: int, y: int) -> int:
	"""Useful function to multiply two numbers."""
	return x * y

tools = [
	agents.FunctionTool.from_defaults(add),
	agents.FunctionTool.from_defaults(multiply),
]

agent = AgentFunctionCalling(
	tools=tools, timeout=120, verbose=True,
)

In [ ]:
# user_query = "Hello"
user_query = "What is (2123 + 2321) * 312?"
# user_query = "Tell me a joke"
result = await agent.run(user_query=user_query)

pprint(result)

#### REACT

# App

In [ ]:
from features.agents.car import apis as apis_car
from features.rag import apis as apis_rag

In [ ]:
queries = [
	"What are notes before driving car",
	# "Please provide a summary of the content found on page 200 of the manual",
	"How can parents prevent children from accidentally opening doors or windows while driving?",

	"Fold the mirrors",
	"Unfold the mirrors",
	"Direct air to feet only",
	"Direct air to face and feet",
	
	"Is the car locked",
	"Is the car trunk opened?",
	"Open the car trunk",
 
	"It's too hot",
	"I'm feeling sleepy",
	"The radio is too loud",
	"Can you turn on the AC?",
	"The windshield is getting foggy",
	"The headlights seem dim",
]

In [ ]:
prompts_agent_car = settings.prompts_agent_car

class EvUserQueryCategorized(wfs.Event):
	user_query_category: t.UserQueryCategory
#*------------------------------------------------------------------------------
class EvFlowStartedRag(wfs.Event):
	pass

class EvFlowDoneRag(wfs.Event):
	pass

#*------------------------------------------------------------------------------
class EvFlowStartedControl(wfs.Event):
	pass

# class EvFlCtrlProcessTaskStarted(wfs.Event):
# 	task: str
# 	id: str

# class EvFlCtrlProcessTaskDone(wfs.Event):
# 	result: str
# 	id: str

class EvFlowDoneControl(wfs.Event):
	pass
#*------------------------------------------------------------------------------
class EvFlowStartedGeneral(wfs.Event):
	pass

class EvFlowDoneGeneral(wfs.Event):
	pass

#*------------------------------------------------------------------------------
class EvFlowDone(wfs.Event):
	pass

#*------------------------------------------------------------------------------
class EvHumanFeedbackDone(wfs.Event):
  human_feedback: dict[str, Any]

class EvHumanSatisfied(wfs.Event):
	pass

#*------------------------------------------------------------------------------
class MyWorkflow(wfs.Workflow):
	@wfs.step()
	async def categorize_user_query(
		self, ctx: wfs.Context, ev: wfs.StartEvent,
	) -> EvUserQueryCategorized:
		user_query = ev.get("user_query", "")

		user_query_category = await apis_car.categorize_user_query(user_query)
		rp_print(user_query_category)
		
		await ctx.set("user_query", user_query)
		await ctx.set("user_query_category", user_query_category)
 
		return EvUserQueryCategorized(user_query_category=user_query_category)
	
	@wfs.step()
	async def start_flow(
		self, ctx: wfs.Context, ev: EvUserQueryCategorized
	) -> EvFlowStartedRag | EvFlowStartedControl | EvFlowStartedGeneral:
		user_query_category: t.UserQueryCategory = await ctx.get("user_query_category")
		
		flow_mapping = {
			"car_manual": ("rag", EvFlowStartedRag),
			"car_control": ("control", EvFlowStartedControl),
			"general": ("general", EvFlowStartedGeneral),
		}

		if user_query_category in flow_mapping:
			task, event_class = flow_mapping[user_query_category]
			
			flow_info = {
				"activated": True,
				"task": task
			}
			await ctx.set("flow_info", flow_info)
			return event_class()
	
	@wfs.step()
	async def run_flow_rag(
		self, ctx: wfs.Context, ev: EvFlowStartedRag,
	) -> EvFlowDoneRag:
		flow_info = await ctx.get("flow_info")
		user_query = await ctx.get("user_query")

		result = await apis_rag.do_querying(user_query=user_query, mode="achat")

		if flow_info:
			flow_info["completed"] = True
			flow_info["result"] = result
   
		await ctx.set("flow_info", flow_info)

		return EvFlowDoneRag()

	@wfs.step()
	async def run_flow_general(
		self, ctx: wfs.Context, ev: EvFlowStartedGeneral,
	) -> EvFlowDoneGeneral:
		flow_info = await ctx.get("flow_info")
		user_query = await ctx.get("user_query")

		result = await apis_car.do_general(user_query=user_query, mode="achat")

		if flow_info:
			flow_info["completed"] = True
			flow_info["result"] = result
   
		await ctx.set("flow_info", flow_info)

		return EvFlowDoneGeneral()

	@wfs.step()
	async def run_flow_control(
		self, ctx: wfs.Context, ev: EvFlowStartedControl,
	) -> EvFlowDoneControl:
		flow_info = await ctx.get("flow_info")
		user_query = await ctx.get("user_query")

		# Get tasks
		tasks = await apis_car.separate_tasks(user_query=user_query)
		rp_print(tasks)

		if not tasks:
			if flow_info:
				flow_info["tasks"] = {}
				flow_info["n_tasks"] = 0
				flow_info["result"] = "No tasks were identified in your request."
				await ctx.set("flow_info", flow_info)
			return EvFlowDoneControl()

		# Initialize tasks map with IDs
		task_map = {}
		for task in tasks:
			task_id = str(utils.uuid_utils.generate_uuid4())
			task_map[task_id] = {
				"task": task,
				"result": None
			}

		flow_info["tasks"] = task_map
		flow_info["n_tasks"] = len(tasks)
		await ctx.set("flow_info", flow_info)

		# Process all tasks concurrently
		async def process_task(task_id: str, task: str) -> tuple[str, str]:
			try:
				result = await apis_car.do_controlling(user_query=task, mode="achat")
				return task_id, result
			except Exception as e:
				logger.error(f"Error processing task {task_id}: {str(e)}")
				return task_id, f"Error: {str(e)}"

		# Create and gather all task coroutines
		coroutines = [
			process_task(task_id, task_info["task"]) 
			for task_id, task_info in task_map.items()
		]
		
		# Execute all tasks concurrently
		results = await asyncio.gather(*coroutines)

		# Update flow info with results
		for task_id, result in results:
			flow_info["tasks"][task_id]["result"] = result

		# Combine all results
		result_texts = [
			task_info["result"] 
			for task_info in flow_info["tasks"].values() 
			if task_info["result"]
		]
		
		flow_info["result"] = "\n".join(result_texts)
		await ctx.set("flow_info", flow_info)

		return EvFlowDoneControl()
			
	@wfs.step()
	async def complete_flow(
		self, ctx: wfs.Context, ev: EvFlowDoneRag | EvFlowDoneControl | EvFlowDoneGeneral
	) -> EvFlowDone:
		flow_info = await ctx.get("flow_info")

		if flow_info:
			flow_info["confirmed"] = True
		await ctx.set("flow_info", flow_info)
  
		return EvFlowDone()

	@wfs.step()
	async def human_feedback(
		self, ctx: wfs.Context, ev: EvFlowDone,
	) -> EvHumanFeedbackDone:
		human_feedback = {
			"feedback": "OK!",
			"retry": False,
		}

		await ctx.set("human_feedback", human_feedback)
		return EvHumanFeedbackDone(human_feedback=human_feedback)
	
	@wfs.step()
	async def retry(
		self, ctx: wfs.Context, ev: EvHumanFeedbackDone
	) -> EvHumanSatisfied | EvUserQueryCategorized:
		human_feedback = await ctx.get("human_feedback")
  
		if human_feedback["retry"] == True:
			return EvUserQueryCategorized(
				user_query_category=await ctx.get("user_query_category")
			)
		else:
			return EvHumanSatisfied()
 
	@wfs.step()
	async def stop(
		self, ctx: wfs.Context, ev: EvHumanSatisfied,
	) -> wfs.StopEvent:
		
		rp_print(ctx.data)
  
		flow_info: dict = await ctx.get("flow_info")
		
		if flow_info:
			result = flow_info["result"]
		
		return wfs.StopEvent(result=result)

async def main():
	workflow = MyWorkflow(timeout=60, verbose=True)

	# user_query = queries[0]
	# user_query = "It's too hot"
	# user_query = "Is the car trunk opened?"
	# user_query =  "Is the car locked?"
	# user_query = "Yes"
	# user_query = "Activate the AC mode. Increase front wiper speed"
	user_query = "Is the car locked? Is the car trunk opened?"
	# user_query = "Is the car locked? Is the car trunk opened?. Increase front wiper speed"
	# user_query = "Decrease front wiper speed"
	result = await workflow.run(user_query=user_query)

wfs.draw_all_possible_flows(MyWorkflow)

asyncio.run(main())

### Test

In [ ]:
user_query = "Is the car trunk opened?"

await apis_car.do_controlling(
	user_query=user_query,
	mode="achat",
)

In [ ]:
queries = [
	"Is the car trunk opened?",
	"What are the main safety procedures to follow before driving the vehicle?",
]

user_query = queries[0]

user_query_category = await apis_car.categorize_user_query(user_query)
rp_print(user_query_category)

In [ ]:
queries_rag = [
	"What are the main safety procedures to follow before driving the vehicle?",
	"What is the correct driving posture recommended in the manual?",
	"What are the differences between Apple CarPlay and Android Auto integration?",
	"How to turn the audio system on and off?",
	"What is the complete procedure for securing child restraint systems?",
	"Summarize the content of page 200 in the manual"
]

user_query = queries_rag[0]

# await components.retriever_car_manual.aretrieve(user_query)

pprint(await apis_rag.do_querying(user_query=user_query, mode="achat"))

In [ ]:
user_query = "Activate the AC mode. Increase front wiper speed. Help me unlock doors."
user_query = "Increase front wiper speed"

await apis_car.separate_tasks(user_query=user_query)

### Timing

In [ ]:
import random

user_query = queries[random.randint(0, len(queries) - 1)]

user_query_category = await utils.time_it(
	lambda: apis_car.categorize_user_query(user_query), 
	name="Categorize user query",
	# n_times=10,
)

In [ ]:
user_query_RAG = queries[0]

result = await utils.time_it(
	lambda: apis_rag.do_querying(user_query=user_query_RAG, mode="chat"), 
	name="Run flow RAG",
	n_times=10,
)

In [ ]:
user_query_control = queries[2]

result = await utils.time_it(
	lambda: await apis_car.do_controlling(user_query=user_query_control, mode="achat"), 
	name="Run flow control",
	# n_times=10,
)